## Part 1: Preprocessing

In [73]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [74]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [75]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [76]:
# Create a list of at least 10 column names to use as X data
X = [
    'Age',
    'BusinessTravel',
    'OverTime',
    'HourlyRate',
    'DistanceFromHome',
    'Education',
    'EducationField',
    'StockOptionLevel',
    'YearsAtCompany',
    'EnvironmentSatisfaction',
]


# Create X_df using your selected columns

X_df = attrition_df[X]

# Show the data types for X_df
print(X_df.dtypes)


Age                         int64
BusinessTravel             object
OverTime                   object
HourlyRate                  int64
DistanceFromHome            int64
Education                   int64
EducationField             object
StockOptionLevel            int64
YearsAtCompany              int64
EnvironmentSatisfaction     int64
dtype: object


In [77]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [78]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

,count
OverTime,
No,799
Yes,303


In [79]:
X_train['BusinessTravel'].value_counts()

,count
BusinessTravel,
Travel_Rarely,788
Travel_Frequently,201
Non-Travel,113


In [80]:
# Ensuring that categorical values are properly encoded before applying the StandardScaler Instance

# Create a OneHotEncoder for the OverTime column
ot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')

# Fit the encoder to the training data
ot_encoder.fit(np.array(X_train['OverTime']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data

X_train['ot_encoded'] = ot_encoder.transform(np.array(X_train['OverTime']).reshape(-1, 1))
X_test['ot_encoded'] = ot_encoder.transform(np.array(X_test['OverTime']).reshape(-1, 1))

print(X_train['ot_encoded'].head())
print(X_test['ot_encoded'].head())

591     0.0
267     0.0
1236    1.0
788     0.0
1224    0.0
Name: ot_encoded, dtype: float64
471     0.0
1090    0.0
441     0.0
979     0.0
1085    0.0
Name: ot_encoded, dtype: float64


In [81]:
# Continuation of categorical values that are properly encoded before the StandardScaler Instance

# Create and fit the Label Encoder for the BusinessTravel column
from sklearn.preprocessing import LabelEncoder

bt_label_encoder = LabelEncoder()

bt_label_encoder.fit(X_train['BusinessTravel'].values)

# Create two new variables by applying the encoder
# to the training and testing data

X_train['BusinessTravel'] = bt_label_encoder.transform(X_train['BusinessTravel'].values)
X_test['BusinessTravel'] = bt_label_encoder.transform(X_test['BusinessTravel'].values)

# Display the data

print(X_train['BusinessTravel'].head(10))

591     2
267     0
1236    2
788     2
1224    2
1384    2
1171    2
25      2
1392    2
480     1
Name: BusinessTravel, dtype: int64


In [82]:
# Identifying additional categorical values due to issues of running the StandardScaler below
categorical_values = X_train.select_dtypes(include=['object']).columns
categorical_values

Index(['OverTime', 'EducationField'], dtype='object')

In [83]:
# OneHotEncoding the listed values above
categorical_columns = [col for col in X_train.columns if X_train[col].dtype == 'object']
categorical_columns

X_train = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)

In [84]:
# Ensuring X_test has the same columns as X_train
for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0

# Reordering the X_test columns to match the X_train
X_test = X_test[X_train.columns]

# Displaying testing columns for verification
X_train.head()

,Age,BusinessTravel,HourlyRate,DistanceFromHome,Education,StockOptionLevel,YearsAtCompany,EnvironmentSatisfaction,ot_encoded,OverTime_Yes,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree
591,33,2,69,16,3,0,3,1,0.0,False,False,True,False,False,False
267,25,0,85,5,2,2,6,2,0.0,False,True,False,False,False,False
1236,36,2,96,13,5,3,2,2,1.0,True,False,True,False,False,False
788,28,2,59,10,3,0,8,3,0.0,False,False,False,False,True,False
1224,26,2,62,17,4,3,3,4,0.0,False,False,False,True,False,False


In [85]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler = scaler.fit(X_train)

# Scale the training and testing data

scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [86]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data

department_encoded = department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data

dept_encoded_train = department_encoder.transform(np.array(y_train[['Department']]))
dept_encoded_test = department_encoder.transform(np.array(y_test[['Department']]))

print(dept_encoded_train[0:5])

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [87]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
attrition_encoded = attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data

attri_encoded_train = attrition_encoder.transform(np.array(y_train[['Attrition']]))
attri_encoded_test = attrition_encoder.transform(np.array(y_test[['Attrition']]))

print(attri_encoded_train[0:6])

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


## Create, Compile, and Train the Model

In [88]:
# Find the number of columns in the X training data
from tensorflow.keras.layers import Input, Dense

n_columns = X_train.shape[1:]

# Create the input layer
input_layer = layers.Input(shape=(n_columns))

# Create at least two shared layers
shared_neural_layer_1 = layers.Dense(64, activation='relu')(input_layer)
shared_neural_layer_2 = layers.Dense(128, activation='relu')(shared_neural_layer_1)

In [89]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = Dense(32, activation='relu', name='department_hidden')(shared_neural_layer_2)

# Create the output layer
department_output = Dense(3, activation='softmax', name='department_output')(department_hidden)


In [90]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = Dense(32, activation='relu', name='attrition_hidden')(shared_neural_layer_2)

# Create the output layer
attrition_output = Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)


In [91]:
# Create the model
neural_network_model = Model(inputs=input_layer, outputs=[department_output, attrition_output], name='neural_network_model')

# Compile the model
neural_network_model.compile(optimizer='adam',
                             loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
                             metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
neural_network_model.summary()

Model: "neural_network_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 15)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │          1,024 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 128)            │          8,320 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,765 (69.39 KB)

 Trainable params: 17,765 (69.39 KB)

 Non-trainable params: 0 (0.00 B)

In [92]:
# Train the model
neural_network_model.fit(X_train, {'department_output': dept_encoded_train, 'attrition_output': attri_encoded_train},
                         epochs=100, batch_size=35, verbose=1, validation_split=0.2)


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - attrition_output_accuracy: 0.8484 - department_output_accuracy: 0.3560 - loss: 5.0558 - val_attrition_output_accuracy: 0.8145 - val_department_output_accuracy: 0.6471 - val_loss: 1.4303
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8048 - department_output_accuracy: 0.6402 - loss: 1.4790 - val_attrition_output_accuracy: 0.8235 - val_department_output_accuracy: 0.6561 - val_loss: 1.2319
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8407 - department_output_accuracy: 0.6596 - loss: 1.1993 - val_attrition_output_accuracy: 0.8235 - val_department_output_accuracy: 0.6244 - val_loss: 1.1909
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8156 - department_output_accuracy: 0.6868 - loss: 1.1883 - val_attrition_output_accuracy: 0.8235 - val_department_output_accuracy: 0.6606 - val_loss: 1.2663
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4

In [93]:
# Evaluate the model with the testing data
test_results = neural_network_model.evaluate(scaled_X_test, {'department_output': dept_encoded_test, 'attrition_output': attri_encoded_test})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.5542 - department_output_accuracy: 0.5034 - loss: 1.8795 


[1.8944544792175293, 0.5353260636329651, 0.532608687877655]

In [94]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[2]}")
print(f"Attrition Accuracy: {test_results[1]}")

Department Accuracy: 0.532608687877655
Attrition Accuracy: 0.5353260636329651


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I do not think that accuracy would be the best metric to use considering that the outputs above are similar in their respective sense. I believe there has to be some level of variation to compare when running on a virtual machine.
2. I chose to run the 'softmax' activation function due to its probability of ensuring that all values are added to the sum of 1 between 0 and 1. It gives the outputs a little more flexibility instead of calculating through the normalized binary route.
3. One way to improve upon the model is acquiring real-world data that highlights realistic outcomes. Doing this would create a larger diversity of accuracy modeling when evaluating the additional dense layers.